## Just a bunch of trash code that could be useful in the future

#### Simulation class with STDP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Simulation:
    def __init__(self) -> None:
        # Time Parameters
        self.dt = 0.01
        self.T = 200
        self.time = np.arange(0, self.T, self.dt)

        # Initial conditions
        self.V = np.zeros(len(self.time))
        self.W = np.zeros(len(self.time))
        self.V[0] = -60
        self.W[0] = 0.014

        self.model = Morris_Lecar()

    # def __init__(self, dt, T, time, V, W, v, w) -> None:
    #     self.dt = dt
    #     self.T = T
    #     self.time = time

    #     # Initial conditions
    #     self.V = V
    #     self.W = W
    #     self.V[0] = v
    #     self.W[0] = w

    #     self.model = Morris_Lecar()
    
    def loop(self):
        for self.t in range(1, len(self.time)):
            m = self.model.m_inf(self.V[self.t-1])
            w = self.W[self.t-1]
            dV = (self.model.I_ext - self.model.gCa * m * (self.V[self.t-1] - self.model.VCa) - 
                  self.model.gK * w * (self.V[self.t-1] - self.model.VK) - self.model.gL * (self.V[self.t-1] - self.model.VL)) * self.dt
            dW = (self.model.w_inf(self.V[self.t-1]) - w) / self.model.tau_w(self.V[self.t-1]) * self.dt
            self.V[self.t] = self.V[self.t-1] + dV
            self.W[self.t] = self.W[self.t-1] + dW

        # Plotting the results
        plt.figure(figsize=(10, 5))
        plt.plot(self.time, self.V, label='Membrane potential (V)')
        plt.xlabel('Time (ms)')
        plt.ylabel('Membrane potential (mV)')
        plt.title('Morris-Lecar Model')
        plt.legend()
        plt.show()
        
    def STDP(self):
        weight_change = self.model.STDP(delta_t=10)
        print(f"Weight change due to STDP: {weight_change}.")

In [ ]:
sim = Simulation()
sim.loop()

In [ ]:
# Example test
numN = 10
dt = 0.01
time = 100
snn = SNN(num_neurons=numN, dt=dt)
snn.run(duration=time)

# Plotting the results
plt.figure(figsize=(10, 5))
for i, neuron in enumerate(snn.neurons):
    plt.plot(np.arange(0, time, dt), neuron.V_vals, label=f'Neuron {i+1}')
# plt.ylim(-36, -35.5)
plt.xlabel('Time (ms)')
plt.ylabel('Membrane potential (mV)')
plt.title('SNN with Morris-Lecar Neurons and STDP')
plt.legend()
plt.show()

In [ ]:
# Training function
def train(model, X_train, y_train, epochs=10, lr=0.01):
    # Initialize lists to store the values
    loss_history = []
    accuracy_history = []
    spike_record = []
    membrane_potential = []

    for epoch in range(epochs):
        epoch_loss = 0
        correct_predictions = 0

        for x, y in zip(X_train, y_train):
            # Forward pass
            output = model.forward(x)
            
            # Ensure y is an array
            if np.isscalar(y):
                y = np.array([y])
            
            # Compute loss
            loss = np.mean((output - y) ** 2)
            epoch_loss += loss
            
            # Calculate accuracy
            predicted_class = np.argmax(output)
            true_class = np.argmax(y)
            if predicted_class == true_class:
                correct_predictions += 1
            
            # Record spike and membrane potential
            spike_record.append(model.neuron.V_vals[-1])  # Assuming last value is the spike
            membrane_potential.append(model.neuron.V)
            
            # Backward pass (gradient descent)
            grad_output = 2 * (output - y) / len(y)
            grad_fc2_weights = np.outer(model.neuron.V, grad_output)
            grad_hidden = np.dot(grad_output, model.fc2_weights.T)
            grad_fc1_weights = np.outer(x, grad_hidden)
            
            # Update weights
            model.fc2_weights -= lr * grad_fc2_weights
            model.fc1_weights -= lr * grad_fc1_weights

        # Calculate average loss and accuracy for the epoch
        avg_loss = epoch_loss / len(X_train)
        accuracy = correct_predictions / len(X_train)
        
        # Append to history lists
        loss_history.append(avg_loss)
        accuracy_history.append(accuracy)

        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss}, Accuracy: {accuracy}')

    np.save("Results/NormalSNN/loss_history.npy", loss_history)
    np.save("Results/NormalSNN/accuracy_history.npy", accuracy_history)
    np.save("Results/NormalSNN/spike_record.npy", spike_record)
    np.save("Results/NormalSNN/membrane_potential.npy", membrane_potential)  
    return loss_history, accuracy_history, spike_record, membrane_potential